# Graphical User Interface (GUI)

In [1]:
from tkinter import *
import pickle
import nltk
from nltk.tag.perceptron import PerceptronTagger
import sys
import numpy as np

from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import difflib
import csv
import sys
import ast
import random
from random import randrange
import re
from nltk.util import ngrams


csv.field_size_limit(1000000)


131072

In [2]:
withinteraction=1
habit=0

#1.. with
#0.. without

In [3]:
if habit==0:
    with open("allpre.txt", "rb") as fp:   # Unpickling
        allprescriptions = pickle.load(fp)

    with open("allrealpre.txt","rb") as fp:
        allrealprescriptions = pickle.load(fp)

    with open("fold_data.txt","rb") as fp:
        fold_data=pickle.load(fp)

else:
    with open("allpre_h.txt", "rb") as fp:   # Unpickling
        allprescriptions = pickle.load(fp)

    with open("allrealpre_h.txt","rb") as fp:
        allrealprescriptions = pickle.load(fp)

    with open("fold_data_h.txt","rb") as fp:
        fold_data=pickle.load(fp)


In [4]:
if 'syndata' in globals():
    del syndata
syndata=[]
for i in range(len(fold_data)):
    syndata.extend(fold_data[i])

In [5]:
print(syndata[0:10])

[['ACEBUTOLOL', '400', 'mg', 'start prn', 'OD', 'PO', 'Max 1', '', '', 'ACEBUTOLOOL', '400', 'mg', 'start prn', 'OD', 'PO', 'Max 11', '', '', [6, 0, 2, 7, 3, 4, 5, 1], 'ACEBUTOLOOL start prn OD PO Max 11 400 mg'], ['ACICLOVIR', '', '', 'start PRN', '5 times a day', 'skin', '', '', '5-10 days', 'ACICLOVOR', '', '', 'start PRN', '5 times a dam', 'skin', '', '', '5-10 days', [4, 3, 7, 1, 5, 0, 6, 2], 'skin 5 times a dam 5-10 days ACICLOVOR start PRN'], ['ACARBOSE', '50', 'mg', 'starting prn', 'every day', 'PO', 'Max per dose 200 mg 3 times a day', '', '6-8 weeks', 'ACARBOS', '0', 'mq', 'starting prn', 'every day', 'PO', 'ax per dose 2000 mg 3 taes a day', '', '6-8 weeks', [4, 5, 6, 1, 7, 0, 2, 3], 'PO ax per dose 2000 mg 3 taes a day 0 mq 6-8 weeks ACARBOS starting prn every day'], ['ACEBUTOLOL', '300', 'mg', 'start prn', 'TDS', 'PO', 'Max 1', '', '', 'ACEBUTXLO', '300', 'mg', 'tarb prrn', 'TDS', 'PO', 'Max 1', '', '', [0, 5, 2, 6, 1, 3, 4, 7], 'ACEBUTXLO Max 1 tarb prrn 300 mg TDS PO'], 

In [6]:
if withinteraction==1:
    alldata=[]
    with open('interactions.csv', 'r',newline='',encoding="utf-8") as csvfile:  
        csvreader = csv.reader(csvfile,quoting=csv.QUOTE_ALL)
        for row in csvreader:
            alldata.append(row)
    if len(alldata)==1:
        alldata=alldata[0]

    interactions={}
    intlist=[]
    for i in range(0,len(alldata),2):
        #string dictionary to dictionary:
        interactions[alldata[i]]=ast.literal_eval(alldata[i+1])
        
        #save all interaction drug names in extra list
        for k,v in interactions[alldata[i]].items():
            intlist.append(k)
    
    random.seed(5)
    #randomly select if a patient already takes 1-10 drugs
    n_drugs=randrange(10)
    curr_patient=[]
    severe_interactions=[]
    moderate_interactions=[]
    while len(curr_patient)<n_drugs:
        drug=randrange(len(intlist))
        if intlist[drug] not in curr_patient:
            curr_patient.append(intlist[drug])
            for keys in interactions.keys():
                if intlist[drug] in interactions[keys].keys():
                    severity=interactions[keys][intlist[drug]][0]
                    if severity=='Severe':
                        if keys not in severe_interactions:
                            severe_interactions.append(keys)
                    elif severity=='Moderate':
                        if keys not in moderate_interactions:
                            moderate_interactions.append(keys)
    print(curr_patient)
    print(severe_interactions)
    print(moderate_interactions)

['Promethazine', 'Aripiprazole', 'Cisplatin', 'Cilostazol', 'Nicorandil', 'Disopyramide', 'Zoledronate', 'Darunavir', 'Naproxen']
['acenocoumarol', 'alteplase', 'apixaban', 'argatrob', 'aceclofenac', 'aspirin', 'acebutolol', 'amifampridine', 'aminophylline', 'amisulpride', 'amphotericin', 'anagrelide', 'apalutamide', 'apomorphi', 'arsenic-trioxide', 'artemeth', 'artenim', 'atazanavir', 'atenolol', 'abemaciclib', 'alfentanil', 'amiodaro', 'atorvastatin', 'avanafil']
['atazanavir', 'acenocoumarol', 'alfuzos', 'alitretinoin', 'alprazolam', 'amlodipine', 'aprepitant', 'aripiprazole', 'atovaquone', 'axitinib', 'alendrona']


In [7]:
interactions['alitretinoin']['Darunavir']

['Moderate',
 'Darunavir is predicted to increase the exposure to alitretinoin. Manufacturer advises adjust alitretinoin dose. Severity of interaction: Moderate Evidence for interaction: Theoretical']

In [8]:
units=[]
for i in range(len(syndata)):
    if syndata[i][11] not in units:
        units.append(syndata[i][11])

In [9]:
type(np.int('12ab'[0]))==int

True

In [10]:
synonyms={}
for i in range(len(allrealprescriptions)):
    for j in range(len(allrealprescriptions[i])):
        if allrealprescriptions[i][j][0] not in synonyms.keys():
            try:
                np.int(allrealprescriptions[i][j][0][0])
            except:
                synonyms[allrealprescriptions[i][j][0]]=allrealprescriptions[i][j][0]
        if allprescriptions[i][j][0]!=allrealprescriptions[i][j][0]:
            if allprescriptions[i][j][0] not in synonyms.keys():
                try:
                    np.int(allprescriptions[i][j][0][0])
                except:
                    synonyms[allprescriptions[i][j][0]]=allrealprescriptions[i][j][0]
                
        

In [11]:
synonymsrev={}
for k,v in synonyms.items():
    for i in v:
        if i not in synonymsrev.keys():
            synonymsrev[i]=[k]
        else:
            synonymsrev[i].append(k)

"N-Grams" of all classes:

Find n-grams always including word1 to predict the next one:
- [word1]
- [word1, word2]
- ...

https://medium.com/@davidmasse8/predicting-the-next-word-back-off-language-modeling-8db607444ba9

In [12]:
tagorder=['DN',['DO','UN'],'ST','FR','RO','MA','MI','DU']
tagorder_all=[]
for j in range(len(syndata)):
    tagorder_all.append([])
    curr_tag=[tagorder[x] for x in syndata[j][18]]
    for i in curr_tag:
        tag_ind=tagorder.index(i)
        if tag_ind>1:
            tag_ind+=1
        if syndata[j][tag_ind]!='':
            class_n=len(syndata[j][tag_ind].split())
            if class_n>1:
                tagorder_all[-1].extend([i for k in range(class_n)])
                continue
            if type(i)==list:
                tagorder_all[-1].extend(i)
                continue
            tagorder_all[-1].append(i)


In [13]:
first_word_class={'DN':0,'DO':0,'UN':0,'ST':0,'MA':0,'MI':0,'DU':0,'FR':0,'RO':0}
for i in range(len(tagorder_all)):
    first_word_class[tagorder_all[i][0]]+=1

all_grams={}

for i in range(len(tagorder_all)):
    curr_tag_classes=tagorder_all[i]
    curr_grams=[]
    curr_grams.extend(ngrams(curr_tag_classes,2)) #bigrams
    curr_grams.extend(ngrams(curr_tag_classes,3)) #trigrams
    curr_grams.extend(ngrams(curr_tag_classes,4)) #4-grams
    curr_grams.extend(ngrams(curr_tag_classes,5)) #5-grams
    for i in curr_grams:
        class_concat=" ".join(i)
        if class_concat in all_grams.keys():
            all_grams[class_concat]+=1
        else:
            all_grams[class_concat]=1


In [14]:
tag_grams=[]
for i in range(len(tagorder_all)):
    for j in range(2,len(tagorder_all[i])):
        tag_grams.append(tagorder_all[i][:j])
    tag_grams.append(tagorder_all[i])

gram_dict={}
for i in range(len(tag_grams)):
    class_join=' '.join(tag_grams[i])
    if class_join in gram_dict.keys():
        gram_dict[class_join]+=1
    else:
        gram_dict[class_join]=1

In [15]:
tagg=PerceptronTagger(load=False)
tagg.train(allprescriptions)

In [16]:
def word_doun_split(word,tagg):
    currtag=tagg.tag(word)
    justsplit=0
    changed=0
    for n,i in enumerate(word):
        if justsplit==1:
            justplit=0
            continue
        digits=[]
        if i[0].isdigit()==True:
            digits.append(True)
            anyfalse=0
            for j in range(1,len(i)):
                digits.append(i[j].isdigit())
                if i[j] in [".","-"]:
                    digits[-1]=True
                    continue
                if i[j].isdigit()==False:
                    anyfalse=1
            if anyfalse==0:
                continue
            k=len(i)-1
            while digits[k]==False:
                k-=1
            justplit=1
            oldword=word[n]
            word[n]=i[0:k+1]
            word.insert(n+1,i[k+1:])
            tagged=tagg.tag(word)
            if tagged[n+1][1]!='UN' or isvalidprescription(tagged)==1:
                word[n]=i
                word.pop(n+1)
            elif tagged[n+1][1]=='UN':
                changed=1
                break     
    return (changed,word)

In [17]:
def next_word_class(inpstr):
    try:
        _,inpstr=word_doun_split(inpstr.split(),tagg)
    except:
        inpstr=inpstr
    inpstr=' '.join(inpstr)
    if inpstr=='':
        currtag=[('','')]
    currtag=tagg.tag(inpstr.split())
    
    currstr=' '.join([v for k,v in currtag])
    
    tagorder=['DN','DO','UN','ST','FR','RO','MA','MI','DU']
    
    prob={'DN':0,'DO':0,'UN':0,'ST':0,'FR':0,'RO':0,'MA':0,'MI':0,'DU':0}
    
    if currstr=='':
        for i in tagorder:
            prob[i]=first_word_class[i]
        sumprobs=sum(prob.values())
        if sumprobs==0:
            return 0
        else:
            for i in tagorder:
                prob[i]/=sumprobs
        return prob
    else:
        possibletags=[]
        for i in tagorder:
            if i not in currstr:
                possibletags.append(i)
        possibletags.append(currstr.split()[-1])
    
    
    currstr_split=currstr.split()
    for i in range(1,min(5,len(currstr_split)+1)):
        currprob={'DN':0,'DO':0,'UN':0,'ST':0,'FR':0,'RO':0,'MA':0,'MI':0,'DU':0}
        
        for j in possibletags:
            try:
                currprob[j]+=all_grams[" ".join(currstr_split[-i:])+' '+j]
            except:
                currprob[j]+=0
        sumprobs=sum(currprob.values())
        if sumprobs==0:
            continue
        for j in possibletags:
            prob[j]+=currprob[j]/sumprobs
        
    for j in possibletags: 
        prob[j]/=i
    
    
    return prob

In [18]:
bsp="aspirin 75 mg po"

print(tagg.tag(bsp.split()))
next_word_class(bsp)

[('aspirin', 'DN'), ('75', 'DO'), ('mg', 'UN'), ('po', 'RO')]


{'DN': 0,
 'DO': 0,
 'UN': 0,
 'ST': 0.4942569530271679,
 'FR': 0.331416142561322,
 'RO': 0.008725981503720379,
 'MA': 0.060167731216138366,
 'MI': 0.01583914235397505,
 'DU': 0.08959404933767633}

In [19]:
print(interactions['aspirin'])

{'Aceclofenac': ['NotSet', 'Both aspirin and aceclofenac can increase the risk of bleeding.'], 'Acenocoumarol': ['Severe', 'Acenocoumarol is predicted to increase the risk of bleeding events when given with aspirin. Manufacturer advises use with caution or avoid. Severity of interaction: Severe Evidence for interaction: Theoretical'], 'Acetazolamide': ['Severe', 'Acetazolamide increases the risk of severe toxic reaction when given with aspirin (high-dose). Manufacturer makes no recommendation. Severity of interaction: Severe Evidence for interaction: Study'], 'Alendronate': ['Moderate', 'Aspirin (high-dose) is predicted to increase the risk of gastrointestinal irritation when given with alendronate. Manufacturer makes no recommendation. Severity of interaction: Moderate Evidence for interaction: Study'], 'Alteplase': ['Severe', 'Alteplase is predicted to increase the risk of bleeding events when given with aspirin. Manufacturer advises use with caution or avoid. Severity of interaction

In [20]:
next_word_class('aspirin ')

{'DN': 0.3650202172361849,
 'DO': 0.13977642115277888,
 'UN': 0.0,
 'ST': 0.15396812812177912,
 'FR': 0.12201696662173947,
 'RO': 0.15928010782525964,
 'MA': 0.025132799492587014,
 'MI': 0.0025370649330056293,
 'DU': 0.03226829461666535}

In [21]:
#tokenize == only real words without classes!

#all real words
if 'tokenizedallpre' in globals():
    del tokenizedallpre
tokenizedallpre=[]
tokenizedallwords_real=[]

for i in range(len(allrealprescriptions)):
    tokenizedallpre.append([])
    for j in range(len(allrealprescriptions[i])):
        tokenizedallpre[i].append(allrealprescriptions[i][j][0])
        #for word autocomplete:
        tokenizedallwords_real.append(allrealprescriptions[i][j][0])
        
        
#all noised word characters but then used the realprescription words
if 'tokenizedallwords' in globals():
    del tokenizedallwords
tokenizedallwords=[]

for i in range(len(allprescriptions)):
    for j in range(len(allprescriptions[i])):
        tokenizedallwords.append(list(allprescriptions[i][j][0]))

In [22]:
n = 5
train_data, padded_sents = padded_everygram_pipeline(n, tokenizedallpre)

model = MLE(n) # Lets train a 5-grams maximum likelihood estimation model.
model.fit(train_data, padded_sents)

In [23]:
train_data_chars={}
for i in range(len(tokenizedallwords)):
    currword=tokenizedallwords_real[i]
    for j in range(1,len(tokenizedallwords[i])+1):
        currword_part=''.join(tokenizedallwords[i][:j])
        if currword_part not in train_data_chars.keys():
            train_data_chars[currword_part]={}
            train_data_chars[currword_part][currword]=1
        else:
            if currword not in train_data_chars[currword_part].keys():
                train_data_chars[currword_part][currword]=1
            else:
                train_data_chars[currword_part][currword]+=1

In [24]:
def word_complete(inpword):
    try:
        distr=train_data_chars[inpword]
    except:
        return(inpword)
    if len(distr.keys())==1:
        return(list(distr.keys())[0])
    else:
        overall=np.sum(list(distr.values()))
        for k,v in distr.items():
            distr[k]=v/overall
        rand_nr=random.random()
        currd=0
        for val,k in enumerate(list(distr.keys())):
            if rand_nr<=currd+distr[k]:
                return(k)
            else:
                currd+=distr[k]
    return(distr)

In [25]:
print(model.counts['aspirin']) # i.e. Count('language')
print(model.counts[['aspirin']]['75']) # i.e. Count('is'|'language')
print(model.counts[['aspirin', '75','mg']]['per']) # i.e. Count('never'|'language is')

print(model.generate(num_words=1,text_seed=['aspirin','75']))
print(model.generate(num_words=3,text_seed=['aspirin']))
print(model.generate(num_words=1,text_seed=['aspirin','75','mg','start']))
print(model.score('qqqh', 'aspirin'.split()))  # P('is'|'language')
print(model.score('5pm', 'aspirin 75 mg starting'.split()))  # P('never'|'language is')

486
18
0
mg
['with', 'metoclopramide', '1']
8pm
0.0
0.5


In [26]:
bsp="aspirin 75 mg starting 5pm o.d"
print(bsp.split()[-3:])
new=model.generate(num_words=1,text_seed=bsp.split())
print(new)


['starting', '5pm', 'o.d']
po


In [27]:
print(model.score(new,bsp.split()[-5:]))

tagg.tag(bsp.split())[-1][1]

0


'FR'

In [28]:
def isvalidprescription(prescrip):
    tagorder=['DN','DO','UN','ST','FR','RO','MA','MI','DU']
    #0.. valid
    #1.. invalid
    
    pos={}
    for j in tagorder:
        pos[j]=[]

    bsp=prescrip
    for i in range(len(bsp)):
        pos[bsp[i][1]].append(i)

    noprescription=0
    for i in tagorder:
        if len(pos[i])>1 and len(pos[i])!=len(range(pos[i][0],pos[i][-1]+1)):
            noprescription=1
    return(noprescription)

In [29]:
currtag=tagg.tag("75 mg aspirin".split())
print(sum([ta=='DN' for nm,ta in currtag]))

1


In [30]:
def prescription_prediction(inpstr,newwords):
    words=inpstr.strip().split()
    count=model.counts[words[-1]]
    if count==0:
        return []
    if inpstr[-1]!=' ':
        inpstr=inpstr+' '
    
    final_output=[]
    for i in range(newwords):
        currgen_comb=inpstr
        m=0
        while m<15: #max wordnumber of the extension
            n=0
            output=[]
            class_prob=next_word_class(currgen_comb)
            if class_prob==0:
                break
            score=[]
            while n<10:
                currgen=model.generate(num_words=1,text_seed=currgen_comb.split())
                if currgen[0]=='<':
                    n+=1
                    continue
                if currgen in output:
                    n+=1
                    continue
                currtag=tagg.tag(currgen_comb.split()+[currgen])
                #check if probability!=0, last letter!=number,( at least one DN tag)
                if class_prob[currtag[-1][1]]==0 or currtag[-1][0][-1].isdigit()==True:# or sum([ta=='DN' for nm,ta in currtag])==0:
                    n+=1
                    continue
                
                output.append(currgen)
                n+=1
            if len(output)==0:
                break
            elif len(output)==1:
                currgen_comb=currgen_comb+output[0]+' '
            else:
                rand_index=random.randrange(len(output))
                currgen_comb=currgen_comb+output[rand_index]+' '
            m+=1
        if currgen_comb.strip() not in final_output and inpstr.strip()!=currgen_comb.strip():
            final_output.append(currgen_comb.strip())
    return final_output
    

In [31]:
prescription_prediction('aspirin 900 mg',6)

['aspirin 900 mg pr',
 'aspirin 900 mg once starting prn po',
 'aspirin 900 mg starting prn once a month max visual acuity is achieved or there are no signs',
 'aspirin 900 mg max',
 'aspirin 900 mg starting prn po t.i.d']

In [32]:
def diff(word1,word2):
    difference=difflib.ndiff(word1.lower(),word2.lower())
    plus=0
    minus=0
    for i in difference:
        if i[0]=='-':
            minus+=1
        elif i[0]=='+':
            plus+=1
    return [minus,plus]

In [36]:
def word_prediction(inpstr,tagg,currentsuggestions,newwords):
    word=inpstr.split()
    
    [changed,word]=word_doun_split(word,tagg)
    if changed==1:
        inpstr=' '.join(word)
    
    changeword=0
    if inpstr[-1]==" ":
        changeword=1
        output=[inpstr]
    if changeword==0:
        if len(word)<=1:
            next_class=next_word_class('')
        else:
            next_class=next_word_class(' '.join(word[:-1])+' ')
        output=[]
        for i in range(10):
            currstr=word_complete(word[-1])
            if currstr==word[-1]:
                break
            output.append(currstr)
            if len(word)<=1:
                currtag=tagg.tag([output[-1]])
            else:
                currtag=tagg.tag(word[:-1]+[output[-1]])
            try:
                if next_class[currtag[-1][1]]==0:
                    continue
            except:
                continue
            if output[-1] in output[:-1]:
                output.pop(-1)
            else:
                for words in output[:-1]:
                    [minus,plus]=diff(output[-1],words)
                    if minus<=1 and minus+plus<=3:
                        output.pop(-1)
    if len(word)==1 or changeword==1:
        word_list=[w for w in output]
        if output==[]:
            word_list=[w for w in word]
    else:
        word_list=[' '.join(word[:-1])+ " "+ w for w in output]
    if word_list==[]:
        return []
    if len(word_list)>1:
        new_prescription=[]
        for i in word_list.copy():
            new_pres=prescription_prediction(i,newwords)
            if new_pres==[]:
                ipos=word_list.index(i)
                word_list.pop(ipos)
                continue
            new_prescription.append(new_pres)
        if (sum(x) for x in new_prescription)==0:
            return []
        
        #save columnwise:
        if len(new_prescription)==1:
            final_new_prescription=[]
            for ij in range(min(newwords,len(new_prescription[0]))):
                if new_prescription[0][ij] not in currentsuggestions:
                    final_new_prescription.append(new_prescription[0][ij])
        elif len(new_prescription)==0:
            return []
        else:
            prescribeorder=[]
            maxlen=max(len(x) for x in new_prescription)
            i=0
            for i in range(maxlen):
                for j in range(len(new_prescription)):
                    try:
                        prescribeorder.append(new_prescription[j][i])
                    except:
                        continue
            
            final_new_prescription=[]
            i=0
            while len(final_new_prescription)<newwords and i<30:
                try:
                    if isvalidprescription(tagg.tag(prescribeorder[i].split()))==1:
                        i+=1
                        continue
                    if prescribeorder[i] not in currentsuggestions:
                        final_new_prescription.append(prescribeorder[i])
                except:
                    if i==len(prescribeorder)-1:
                        break
                i+=1
    else:
        new_prescription=prescription_prediction(word_list[0],newwords)
        final_new_prescription=[]
        i=0
        while len(final_new_prescription)<newwords and i<30:
            if i==len(new_prescription):
                break
            if isvalidprescription(tagg.tag(new_prescription[i].split()))==1:
                i+=1
                continue
            if new_prescription[i] not in currentsuggestions:
                final_new_prescription.append(new_prescription[i])
            i+=1
    
    inp_split_test=inpstr.split()
    changed=0
    for val,j in enumerate(inp_split_test[:-1]):
        try:
            syn=synonyms[j]
            inp_split_test[val]=syn
            if syn!=j:
                changed=1
        except:
            changed+=0
    if changed==1:
        new_pres=' '.join(inp_split_test)
        if new_pres not in final_new_prescription and new_pres not in currentsuggestions:
            final_new_prescription.insert(0,new_pres)
            if len(final_new_prescription)>newwords:
                final_new_prescription=final_new_prescription[:newwords]

    return final_new_prescription

In [34]:
severe_color="#F1948A"
moderate_color="#F7DC6f"


In [38]:
class AutoEntry(Entry):
    def __init__(self,*args, **kwargs):
        Entry.__init__(self, *args, **kwargs)
        self.var = self["textvariable"]
        if self.var == '':
            self.var = self["textvariable"] = StringVar()

        self.var.trace('w', self.changed)
        self.bind("<Right>", self.selection)
        self.bind("<Up>", self.up)
        self.bind("<Down>", self.down)
        self.bind("<Button-1>",self.close)
        self.lb_up = False
        self.tagg=tagg
        self.lastlen=0
        self.severe_int=[]
        self.moderate_int=[]
        self.interaction=0 #0.. don't show interactions
        self.severity=[0,0,0,0,0,0]
        self.sev_list=[]
        self.mod_list=[]

    def changed(self, name, index, mode):  
        if len(self.var.get()) <=2:
            if self.lb_up==True:
                self.lb.destroy()
                self.lb_up=False
            return
        else:
            change=0
            selfvar=self.var.get()
            if self.lastlen>len(selfvar):
                if selfvar[-1]==' ':
                    change=1
                elif len(re.findall(' ',selfvar))==0:
                    change=0
            self.lastlen=len(self.var.get())
            words=[]
            
            newwords=6
            if self.lb_up==True and change==0:
                varlen=len(selfvar)
                for val,jkl in enumerate(self.lb.get(0,END)):
                    if jkl[:varlen]==selfvar:
                        words.append(jkl)
                        self.severity[len(words)-1]=self.severity[val]
                if len(words)>=6:
                    return
                newwords=max(0,6-len(words))
            oldwordlen=len(words)
            new_word_suggestions=word_prediction(selfvar,self.tagg,words,newwords)
            words.extend(new_word_suggestions)
            newwordlen=len(words)
            if self.interaction==1:
                for i in range(oldwordlen,newwordlen):
                    currtag=tagg.tag(words[i].split())

                    if 'DN' in [v for k,v in currtag]:
                        currdrugname=[k for k,v in currtag if v=='DN']
                        if ' '.join(currdrugname) in self.severe_int:
                            self.severity[i]=2
                        elif ' '.join(currdrugname) in self.moderate_int:
                            self.severity[i]=1
                        else:
                            self.severity[i]=0
                            
            #new_word at pos 1 if only synonyms have been changed:
            if len(new_word_suggestions)>0:
                new_first_len=len(new_word_suggestions[0].split())
                input_len=len(selfvar.split())
                if input_len==new_first_len:
                    syn_word=words[0]
                    syn_sev=self.severity[0]
                    words[0]=words[oldwordlen]
                    self.severity[0]=self.severity[oldwordlen]
                    words[oldwordlen]=syn_word
                    self.severity[oldwordlen]=syn_sev
                        
            if words:            
                if not self.lb_up:
                    self.lb = Listbox(height=6,width=74)
                    self.lb.bind("<Double-Button-1>", self.selection)
                    self.lb.bind("<Right>", self.selection)
                    self.lb.place(x=self.winfo_x(), y=self.winfo_y()+self.winfo_height())
                    self.lb_up = True
                
                self.lb.delete(0, END)
                for val,w in enumerate(words):
                    self.lb.insert(END,w)
                    if self.interaction==1:
                        severi=self.severity[val]
                        if severi==0:
                            self.lb.itemconfig(val,{'bg':'White'})
                            self.lb.itemconfig(val,{'fg':'Black'})
                        elif severi==1:
                            self.lb.itemconfig(val,{'bg':moderate_color})
                            self.lb.itemconfig(val,{'fg':'White'})
                        else:
                            self.lb.itemconfig(val,{'bg':severe_color})
                            self.lb.itemconfig(val,{'fg':'White'})
                self.lb.config(height=newwordlen)
            else:
                if self.lb_up:
                    self.lb.destroy()
                    self.lb_up = False
            self.lastlen=len(self.var.get())
        
    def selection(self, event):

        if self.lb_up:
            self.var.set(self.lb.get(ACTIVE))
            self.lb.destroy()
            self.lb_up = False
            self.icursor(END)
    
    def close(self,event):
        if self.lb_up:
            self.lb.destroy()
            self.lb_up=False
            self.icursor(END)

    def up(self, event):

        if self.lb_up:
            if self.lb.curselection() == ():
                index = '0'
            else:
                index = self.lb.curselection()[0]
            if index != '0':                
                self.lb.selection_clear(first=index)
                index = str(int(index)-1)                
                self.lb.selection_set(first=index)
                self.lb.activate(index) 
    
    def down(self, event):

        if self.lb_up:
            if self.lb.curselection() == ():
                index = '0'
            else:
                index = self.lb.curselection()[0]
            if index != END:                        
                self.lb.selection_clear(first=index)
                index = str(int(index)+1)        
                self.lb.selection_set(first=index)
                self.lb.activate(index) 

    def splitup(self):#,event):
        dns_text.set('')
        dns.config(bg="SystemButtonFace")
        dns.config(fg="SystemButtonFace")
        if self.lb_up:
            self.lb.destroy()
            self.lb_up = False
        Warning.set("")
        for k,v in new_dict.items():
            v.set("")
        inptext=self.var.get()
        if inptext=='':
            return
        word=inptext.lower().split()
        [changed,word]=word_doun_split(word,tagg)
        if changed==1:
            inpstr=' '.join(word)
        currtag=tagg.tag(word)
        if isvalidprescription(currtag)==1:
            Warning.set('Warning: Invalid')
        else:
            classes=[currtag[0][1]]
            newtag=[currtag[0][0]]
            for i in range(1,len(currtag)):
                if currtag[i][1]==currtag[i-1][1]:
                    newtag[-1]=' '.join((newtag[-1],currtag[i][0]))
                else:
                    newtag.append(currtag[i][0])
                    classes.append(currtag[i][1])
            for i,val in enumerate(classes):
                strsyn=newtag[i]
                if str(val) in ["ST","MA","MI"]:
                    new_dict[val].set(" ".join(strsyn.split()[1:]))
                    continue
                new_dict[val].set(strsyn)
                if val=="DN" and self.interaction==1:
                    if word[i] in self.severe_int:
                        dns_text.set(self.sev_list[word[i]])
                        dns.config(bg=severe_color)
                        dns.config(fg="white")
                    elif word[i] in self.moderate_int:
                        dns_text.set(self.mod_list[word[i]])
                        dns.config(bg=moderate_color)
                        dns.config(fg="white")
        return
        
    def clear(self):
        self.var.set('')
        for k,v in new_dict.items():
            v.set("")
        Warning.set("")
        Patient.set("")
        Patient1.set("")
        self.severe_int=[]
        self.moderate_int=[]
        sev.config(bg="SystemButtonFace") 
        mod.config(bg="SystemButtonFace")
        sev.config(fg="SystemButtonFace")
        mod.config(fg="SystemButtonFace")
        var.set(0)
        dns_text.set('')
        dns.config(bg="SystemButtonFace")
        dns.config(fg="SystemButtonFace")
    
    def newpat(self):
        n_drugs=randrange(10)
        curr_patient=[]
        severe_interactions=[]
        moderate_interactions=[]
        severe_int_expl={}
        moderate_int_expl={}
        while len(curr_patient)<n_drugs:
            drug=randrange(len(intlist))
            if intlist[drug] not in curr_patient:
                curr_patient.append(intlist[drug])
                for keys in interactions.keys():
                    if intlist[drug] in interactions[keys].keys():
                        severity=interactions[keys][intlist[drug]][0]
                        if severity=='Severe':
                            if keys not in severe_interactions:
                                severe_interactions.append(keys)
                                severe_int_expl[keys]=interactions[keys][intlist[drug]][1]
                        elif severity=='Moderate':
                            if keys not in moderate_interactions:
                                moderate_interactions.append(keys)
                                moderate_int_expl[keys]=interactions[keys][intlist[drug]][1]
        self.severe_int=severe_interactions
        self.moderate_int=moderate_interactions
        self.sev_list=severe_int_expl
        self.mod_list=moderate_int_expl
        Patient1.set(' '.join(curr_patient))
        if var.get()==0:
            var.set(1)
            self.yes()
            
        #print(self.severe_int)
        #print(self.moderate_int)
        
    def yes(self):
        self.interaction=var.get()
        if self.interaction==1:
            self.newpat()
            sev.config(bg=severe_color) 
            mod.config(bg=moderate_color)
            sev.config(fg="white")
            mod.config(fg="white")
            Patient.set('Patients Current Drugs:')
            Patient1.set(' '.join(curr_patient))
            
            
            moderate.set('Severity: Moderate')
            severe.set('Severity: Severe')
        else:
            sev.config(bg="SystemButtonFace") 
            mod.config(bg="SystemButtonFace")
            sev.config(fg="SystemButtonFace")
            mod.config(fg="SystemButtonFace")
            Patient.set('')
            Patient1.set('')
            moderate.set('')
            severe.set('')
            self.severity=[0,0,0,0,0,0]
            dns_text.set('')
            dns.config(bg="SystemButtonFace")
            dns.config(fg="SystemButtonFace")

fields = ['Drug Name', 'Dose', 'Unit', 'Starting','Frequency','Route','Maximum','Minimum','Duration']

root=Tk()
presc=StringVar()
DN=StringVar()
DO=StringVar()
UN=StringVar()
ST=StringVar()
FR=StringVar()
RO=StringVar()
MA=StringVar()
MI=StringVar()
DU=StringVar()
Warning=StringVar()

new_dict={'DN':DN,'DO':DO,'UN':UN,'ST':ST,'FR':FR,'RO':RO,'MA':MA,'MI':MI,'DU':DU}
new_dict_names={'DN':'Drug Name: ','DO':'Dose: ','UN':'Unit: ','ST':'Starting: ','FR':'Frequency: ','RO':'Route: ','MA':'Maximum: ','MI':'Minimum: ','DU':'Duration: '}
if withinteraction==0:
    root.geometry('500x500')
else:
    root.geometry('600x650')
root.title('Electronic Prescription Project')

row = Frame(root)
lab = Label(row, width=15, text='Prescription:', anchor='w')
ent = AutoEntry(row)#,textvariable=presc)
row.pack(side=TOP, fill=X, padx=5, pady=5)
lab.pack(side=LEFT)
ent.pack(side=RIGHT, expand=YES, fill=X)


row=Frame(root)
mbutton=Button(row,text='Split Up',command=ent.splitup)
mbutton2=Button(row,text='Clear all',command=ent.clear)
if withinteraction==1:
    var = IntVar()
    checkbox = Checkbutton(row,text='Interactions',variable=var, command=ent.yes)
    mbutton3=Button(row,text='New Patient',command=ent.newpat)
    random.seed(5)
row.pack(side=TOP)
mbutton.pack(side=LEFT,padx=10,pady=10)
if withinteraction==1:
    checkbox.pack(side=RIGHT,padx=10,pady=10)
    mbutton3.pack(side=RIGHT,padx=10,pady=10)
mbutton2.pack(side=LEFT,padx=10,pady=10)


Label(root,textvariable=Warning,font=("Courier",20)).pack()

for k,v in new_dict.items():
    row = Frame(root)
    lab = Label(row, width=15, text=new_dict_names[k], anchor='w')
    ent = Entry(row,textvariable=v)
    row.pack(side=TOP, fill=X, padx=5, pady=5)
    lab.pack(side=LEFT)
    ent.pack(side=LEFT, expand=NO, fill=X)
    if withinteraction==1:
        if k=='DN':
            dns_text=StringVar()
            dns=Label(row,wraplength=300,width=300,textvariable=dns_text)
            dns.pack(side=RIGHT)

    
if withinteraction==1:
    Patient=StringVar()
    pat=Label(root,textvariable=Patient,font=("Courier",10))
    pat.pack(side=TOP,padx=10,pady=10)
    Patient1=StringVar()
    pat1=Label(root,wraplength=400,textvariable=Patient1,font=("Courier",10))
    pat1.pack(side=TOP)
    severe=StringVar()
    sev=Label(root,textvariable=severe,font=("Courier",10))
    sev.pack(side=LEFT,padx=20)
    moderate=StringVar()
    mod=Label(root,textvariable=moderate,font=("Courier",10))
    mod.pack(side=LEFT,pady=10)
    
    
    
    
root.mainloop()




In [ ]:

### TODO:

## synonyms ändern auf alle bis auf letzten..
## 

##### TODO:

# change code, that within word suggestions already tagged words will be saved and added
# with new words (with next class predictions!)